In [103]:
import matplotlib.pyplot as plt
import numpy as np

from glob import glob

%matplotlib inline

In [89]:
with open('log.csv', 'w') as f:
    f.write('percentage,infected_mean,infected_median,infected_max,infected_min,fatalities_mean,fatalities_median,fatalities_max,fatalities_min\n')


In [90]:
#First let's group all measurements per compliance percentage together
def get_data(folders):
    infected = []
    fatalities = []
    for f in folders:
        inf = np.load('%s/infected.npy' %f)
        fat = np.load('%s/fatalities.npy' %f)
        infected.append(inf)
        fatalities.append(fat)
        
    return np.asarray(infected), np.asarray(fatalities)
        
    
def get_peaks(data):
    peaks = np.zeros((len(data)))
    for i in range(len(data)):
        peaks[i] = np.mean(data[i])
    
    return peaks
        

def write_log(infected, fatalities, percentage):
    f = open('log.csv', 'a')
    f.write('%s,%f,%f,%f,%f,%f,%f,%f,%f\n' %(percentage,np.mean(infected),np.median(infected),
                                             np.max(infected), np.min(infected), np.mean(fatalities),
                                             np.median(fatalities), np.max(fatalities), np.min(fatalities)))
    f.close()

percentages = ['0.99', '0.95', '0.9', '0.8', '0.7', '0.6', '0.5', '0.25', '0.0']

for percentage in percentages:
    f = glob('low_density/%s/*' %percentage)
    print(len(f))
    print(percentage)
    infected, fatalities = get_data(f)
    
    #np.save('%s_infected.npy' %percentage, infected)
    #np.save('%s_fatalities.npy' %percentage, fatalities)


    infected = get_peaks(infected)
    fatalities = get_peaks(fatalities)

    write_log(infected, fatalities, percentage)

101
0.99
101
0.95
101
0.9
101
0.8
101
0.7
101
0.6
101
0.5
101
0.25
101
0.0


In [124]:
#zero pad
def zero_pad(data):
    max_size = 0
    for d in data:
        if len(d) > max_size:
            max_size = len(d)

    zero_padded = np.zeros((len(data), max_size))        
    for i in range(len(data)):
        data_pad = np.zeros((max_size,))
        data_pad[0:len(data[i])] = data[i]
        zero_padded[i] = data_pad
        
    return zero_padded


percentages = ['0.99', '0.95', '0.9', '0.8', '0.7', '0.6', '0.5', '0.25', '0.0']

for percentage in percentages:

    f = glob('low_density/%s/*' %percentage)
    infected, fatalities = get_data(f)
    
    infected_padded = zero_pad(infected)
    fatalities_padded = zero_pad(fatalities)
    
    np.save('%s_infected.npy' %percentage, np.mean(infected_padded, axis=0))
    np.save('%s_fatalities.npy' %percentage, np.mean(fatalities_padded, axis=0))

In [133]:
max_size = 0
for infect in infected:
    if len(infect) > max_size:
        max_size = len(infect)
        
zero_padded = np.zeros((len(infected), max_size))        
for i in range(len(infected)):
    infected_pad = np.zeros((max_size,))
    infected_pad[0:len(infected[i])] = infected[i]
    zero_padded[i] = infected_pad

In [134]:
np.mean(zero_padded, axis=0).shape

(2747,)

In [135]:
plt.plot(np.mean(zero_padded, axis=0))
plt.show()

In [140]:
pop = 'low_density/'

percentages = ['0.99', '0.95', '0.9', '0.8', '0.7', '0.6', '0.5', '0.25', '0.0']

%matplotlib qt

plt.figure(figsize=(12,8))
plt.subplot(211)

plt.title('effect of compliance to self-isolation on infections. Population density: low')
plt.xlabel('simulation time steps')
plt.ylabel('number of infected')
plt.ylim(0, 2000)

for percentage in percentages:
    data = np.load('%s/%s_infected.npy' %(pop, percentage))
    plt.plot(data, label = '%s, max_cases: %i' %(percentage, np.max(data)))
    
plt.legend()
    
plt.subplot(212)
plt.title('effect of compliance to self-isolation on fatalities. Population density: low')
plt.xlabel('simulation time steps')
plt.ylabel('number of fatalities')
plt.ylim(0, 220)

for percentage in percentages:
    data = np.load('%s/%s_fatalities.npy' %(pop, percentage))
    plt.plot(np.cumsum(np.clip(np.diff(data), a_min=0, a_max=None)), label='%s, max fatalities: %i' %(percentage, np.max(data)))

plt.legend()

plt.tight_layout()
plt.show()
